In [1]:
logName = "traffic_fines_139"

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv("../logdata/%s.csv" % logName, sep=";")
case_id_col = 'Case ID'
timestamp_col = "Complete Timestamp"

/home/coderus/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.shape

(412738, 20)

In [4]:
data[timestamp_col] = pd.to_datetime(data[timestamp_col])

In [5]:
def split_data(data, train_ratio):
    # split into train and test using temporal split

    grouped = data.groupby(case_id_col)
    start_timestamps = grouped[timestamp_col].min().reset_index()
    start_timestamps = start_timestamps.sort_values(timestamp_col, ascending=True, kind='mergesort')
    train_ids = list(start_timestamps[case_id_col])[:int(np.ceil(train_ratio*len(start_timestamps)))]
    train = data[data[case_id_col].isin(train_ids)].sort_values(timestamp_col, ascending=True, kind='mergesort')
    test = data[~data[case_id_col].isin(train_ids)].sort_values(timestamp_col, ascending=True, kind='mergesort')

    return (train, test)

In [6]:
_, test = split_data(data, train_ratio=0.660)

In [7]:
test.shape

(141305, 20)

In [8]:
min_prefix_length = 2
max_prefix_length = 8

In [10]:
test[test[case_id_col]==6625]

,case_id,Activity,activity_duration,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincecasestart,open_cases,remtime
52999,6625,Register,0.000000,2014-07-10 12:56:07,776,7,3,12,0.000000,76,1349.716667
52998,6625,Acceptance_of_requests,16.366667,2014-07-10 13:12:29,792,7,3,13,16.366667,73,1333.350000
52997,6625,Collection_of_documents,89.883333,2014-07-10 14:42:22,882,7,3,14,106.250000,69,1243.466667
52996,6625,Completeness_check,60.900000,2014-07-10 15:43:16,943,7,3,15,167.150000,64,1182.566667
52995,6625,Credit_worthiness_check,1063.616667,2014-07-11 09:26:53,566,7,4,9,1230.766667,70,118.950000
52994,6625,Collateral_check,66.983333,2014-07-11 10:33:52,633,7,4,10,1297.750000,77,51.966667
52993,6625,Credit_committee,45.983333,2014-07-11 11:19:51,679,7,4,11,1343.733333,72,5.983333
52992,6625,Requirements_review,5.983333,2014-07-11 11:25:50,685,7,4,11,1349.716667,70,0.000000


In [9]:
def add_event_nr_case_length(group):
    
    group = group.sort_values(timestamp_col, ascending=True, kind='mergesort')
    group["event_nr"] = range(1, len(group) + 1)
    group['case_length'] = group.shape[0]
    
    return group

In [10]:
test = test.groupby(case_id_col).apply(add_event_nr_case_length)

In [12]:
test.head(10)

article vehicleClass  amount  points Case ID  \
Case ID                                                    
A10418  631       7            A    22.0     0.0  A10418   
        630       7            A    22.0     0.0  A10418   
        629       7            A    22.0     0.0  A10418   
        628       7            A    44.0     0.0  A10418   
        627       7            A    44.0     0.0  A10418   
        626       7            A    44.0     0.0  A10418   
A10420  636       7            A    22.0     0.0  A10420   
        635       7            A    22.0     0.0  A10420   
        634       7            A    22.0     0.0  A10420   
        633       7            A    44.0     0.0  A10420   

                             Activity Resource lastSent notificationType  \
Case ID                                                                    
A10418  631               Create_Fine      541  missing          missing   
        630                 Send_Fine      541  missing          missing   
        629  Insert_Fine_Notification      541        P                P   
        628               Add_penalty      541        P                P   
        627                   Payment      541        P                P   
        626                   Payment      541        P                P   
A10420  636               Create_Fine      541  missing          missing   
        635                 Send_Fine      541  missing          missing   
        634  Insert_Fine_Notification      541        P                P   
        633               Add_penalty      541        P                P   

            dismissal     ...       Complete Timestamp month  weekday  hour  \
Case ID                   ...                                                 
A10418  631       NIL     ...      2007-04-28 22:00:00     4        5    22   
        630       NIL     ...      2007-08-01 22:00:00     8        2    22   
        629       NIL     ...      2007-08-08 22:00:00     8        2    22   
        628       NIL     ...      2007-10-07 22:00:00    10        6    22   
        627       NIL     ...      2008-01-06 23:00:00     1        6    23   
        626       NIL     ...      2008-09-12 22:00:00     9        4    22   
A10420  636       NIL     ...      2007-05-17 22:00:00     5        3    22   
        635       NIL     ...      2007-08-19 22:00:00     8        6    22   
        634       NIL     ...      2007-09-07 22:00:00     9        4    22   
        633       NIL     ...      2007-11-06 23:00:00    11        1    23   

             timesincelastevent  timesincecasestart  event_nr  open_cases  \
Case ID                                                                     
A10418  631                 0.0                 0.0         1        7829   
        630            136800.0            136800.0         2        9643   
        629             10080.0            146880.0         3        9898   
        628             86400.0            233280.0         4       10822   
        627            131100.0            364380.0         5       10400   
        626            359940.0            724320.0         6       14287   
A10420  636                 0.0                 0.0         1        7911   
        635            135360.0            135360.0         2       10346   
        634             27360.0            162720.0         3       10589   
        633             86460.0            249180.0         4       10732   

                remtime  case_length  
Case ID                               
A10418  631  43459200.0            6  
        630  35251200.0            6  
        629  34646400.0            6  
        628  29462400.0            6  
        627  21596400.0            6  
        626         0.0            6  
A10420  636  58924800.0            5  
        635  50803200.0            5  
        634  49161600.0            5  
        633  43974000.0            5  

[10 rows x 21 columns]

In [13]:
for prefix in range(min_prefix_length, max_prefix_length+1):
    print(prefix)
    long_cases = test[test["case_length"] > prefix]
    longer_cases_this_length = long_cases[long_cases["event_nr"] <= prefix]
    longer_cases_this_length.to_csv("../formulas/%s/test_len_%s.csv" % (logName, prefix), index=False)

2
3
4
5
6
7
8


In [14]:
test.groupby("case_length")[case_id_col].nunique()

case_length
3      273
5    23869
6     3438
9       57
Name: Case ID, dtype: int64